# Prepare problem

***Load libraries***

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import cross_val_score

import random
random.seed(123)

***Load the dataset***

In [ ]:
df = pd.read_csv('churn.all2')

# Data exploration & cleaning

Let us first see the column names and the number of unique values in each column. These will give us some idea about the level of measurement we have in each column. 

In [ ]:
for col in df:
    print('{} : {}'.format(col, len(pd.unique(df[col]))))

It seems like ***area_code***, ***intl_plan***, ***voice_mail_plan***, and ***churned*** are categorical variables. The other variables seem to be metric (or continous). The ***state*** and ***phone_number*** variables is probably also categorical.

In [ ]:
df.state

The ***state*** variable is categorical. The dataset is not a very large one. This variable probably will not help us predict anything. There are 51 unique values in this vairable, and we have 5000 cases. So, I will drop this variable.

Note: The common term used for variables are ***features***, but I will keep using ***variables***.

In [ ]:
df.phone_number

Just like the ***state*** variable, the ***phone_number*** variable is also categorical and it will not help us predict anything. Therefore, I will drop that variable as well.

In [ ]:
df.area_code

Similarly, the ***area_code*** is also a categorical variable. There are only three unique values in this variable. It can help us predict the outcome. For that, I need to use OneHotEncoder. But, I do not think it is worth the effort. (Note: I am being lazy here) I will drop this variable as well.

In [ ]:
df = df.drop(['state','area_code','phone_number'], axis = 1)

Let us see how our data look like.

In [ ]:
for col in df.columns:
    print('{}\n{}\n'.format(col, np.array(df[col])))

It looks like we have four string variables (i.e., ***intl_plan***, ***voice_mail_plan***, ***total_eve_charge***, and ***churned***). We will need to convert these variables in to numbers. Let us do it now.

In [ ]:
# intl_plan
intl_plan = np.zeros(df.shape[0])
intl_plan[df.intl_plan == ' yes'] = 1
df.intl_plan = intl_plan

# voice_mail_plan
voice_mail_plan = np.zeros(df.shape[0])
voice_mail_plan[df.voice_mail_plan == ' yes'] = 1
df.voice_mail_plan = voice_mail_plan

# total_eve_charge
df.total_eve_charge = pd.to_numeric(df.total_eve_charge, errors = 'coerce')

# churned
churned = np.zeros(df.shape[0])
churned[df.churned == ' True.'] = 1
df.churned = churned

Now, let us see if we have any NaN values in any of the variables.

In [ ]:
df.isnull().sum()

It seems like we have a missing values in ***total_eve_charge*** and ***total_intl_charge*** variables. We can replace the missing values with the mean.

In [ ]:
df.total_eve_charge.fillna(value = df.total_eve_charge.mean(), inplace = True)
df.total_intl_charge.fillna(value = df.total_intl_charge.mean(), inplace = True)

# Descriptive statistics

Now let us see the correlations between the variables.

In [ ]:
df.corr()

The following four variables are highly correlated with other four variables in the dataset:

1. total_day_charge
2. total_eve_charge
3. total_night_charge
4. total_intl_charge

We do not need these variables. Therefore, I will drop them.    

In [ ]:
df = df.drop(['total_day_charge','total_eve_charge','total_night_charge','total_intl_charge'], axis = 1)

We are left with the following columns.

In [ ]:
df.columns

The distribution of the **number_vmail_messages** variable was not normal or symetric. Most of the clients had zero values for this variable and some of them had non-zero values. Therefore, I converted this variable to a dummy vairable. See the distribution below.

In [ ]:
ax = df.number_vmail_messages.plot.hist()

In [ ]:
df.loc[df.number_vmail_messages > 0, 'number_vmail_messages'] = 1

Now let us see how the continuous variables are distributed.

In [ ]:
df.account_length.plot.hist(bins=50)

In [ ]:
df.total_day_minutes.plot.hist(bins=20)

In [ ]:
df.total_day_calls.plot.hist(bins=20)

In [ ]:
df.total_eve_minutes.plot.hist(bins=20)

In [ ]:
df.total_eve_calls.plot.hist(bins=20)

In [ ]:
df.total_night_minutes.plot.hist(bins=20)

In [ ]:
df.total_night_calls.plot.hist(bins=20)

In [ ]:
df.total_intl_minutes.plot.hist(bins=20)

In [ ]:
df.total_intl_calls.plot.hist(bins=20)

In [ ]:
df.number_customer_service_calls.plot.hist(bins=20)

All the continuous variables have symetric distributions, except the last two (i.e., ***total_intl_calls*** and ***number_customer_service_calls***). It seems like I do not need to transform any of the continuous variables except standardizing them.

Let us see the frequency distributions of the categorical variables.

In [ ]:
for col in ['voice_mail_plan', 'number_vmail_messages', 'churned']:
    print('column name: {}\n{}\n'.format(col, df[col].value_counts()))

All looks good. But, the dataset is unbalanced. There are 707 positive cases and 4293 negative cases. If I train the dataset as is, it will be trained based on negative values. In that case, I will probably have high accuracy, but I may have lower recall or precision. Therefore, I will train the dataset in two different ways. First, I will use the entire dataset and see what I get. Then, I will oversample the positive cases and train the data again.

# Run three algorithms using unbalanced dataset

In [ ]:
# Create the input and output matrices, split them into train and test
df_ = df.values
X = np.array(df_[:,:-1])
y = np.array(df_[:,-1])
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
# Scale the input variables using the standard scaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

**Logistic Regression**

In [ ]:
# Fit a logistic regression model using the train set
logreg = LogisticRegression(solver = 'lbfgs')
logreg.fit(X_train_scaled, y_train)

In [ ]:
# See the score (accuracy)
logreg.score(X_test_scaled, y_test.astype('int'))

In [ ]:
cvs = cross_val_score(LogisticRegression(), X_train_scaled, y_train, cv = 10)
np.mean(cvs), np.std(cvs)

In [ ]:
# Create a confusion matrix and see precision and recall
cm = confusion_matrix(y_test, logreg.predict(X_test_scaled))
print('recall: {}'.format(cm[1,1] / sum(cm[1])))
print('precision: {}'.format(cm[1,1] / (cm[0,1] + cm[1,1])))

In the first run (see above), although accuracy is high, recall and precision are very low.

**Voting Classifier:** ***Logistic regression + decision tree***

In [ ]:
# Use an ensemble model and see if anything changes
# Create the model and fit
voting = VotingClassifier([('logreg', LogisticRegression(C=100, solver='lbfgs')),
                          ('tree', DecisionTreeClassifier(max_depth=3))], voting = 'soft', flatten_transform=False)
voting.fit(X_train_scaled, y_train)

In [ ]:
# See the score (accuracy)
voting.score(X_test_scaled, y_test)

In [ ]:
cvs = cross_val_score(
    VotingClassifier([('logreg', LogisticRegression(C=100, solver='lbfgs')),
                      ('tree', DecisionTreeClassifier(max_depth=3))], voting = 'soft', flatten_transform=False),
                    X_train_scaled, y_train, cv = 10)
np.mean(cvs), np.std(cvs)

In [ ]:
# Create a confusion matrix and see precision and recall
cm = confusion_matrix(y_test, voting.predict(X_test_scaled))
print('recall: {}'.format(cm[1,1] / sum(cm[1])))
print('precision: {}'.format(cm[1,1] / (cm[0,1] + cm[1,1])))

Again, in this second case (see above), the accuracy is high. Precision is also better. But recall is still too low.

**Random Forest**

In [ ]:
# Let us try another ensemble model, random forest.
rf = RandomForestClassifier(n_estimators = 10, warm_start=True)
rf.fit(X_train_scaled, y_train)

In [ ]:
rf.score(X_test_scaled, y_test)

In [ ]:
cvs = cross_val_score(RandomForestClassifier(), X_train_scaled, y_train, cv = 10)
np.mean(cvs), np.std(cvs)

In [ ]:
# Create a confusion matrix and see precision and recall
cm = confusion_matrix(y_test, rf.predict(X_test_scaled))
print('recall: {}'.format(cm[1,1] / sum(cm[1])))
print('precision: {}'.format(cm[1,1] / (cm[0,1] + cm[1,1])))

This is much better. Both recall and precision are high.

# Run three algorithms using balanced dataset

Now, I will make a balanced dataset and train that dataset. 

The next cell creates the balanced dataset. Here is how I do that. First, I select all positive cases. Then, I select a sample from the negative cases, and this sample's size is the same as the number of positive cases in the entire dataset.

In [ ]:
dff = df.loc[df.churned == 0]
dft = df.loc[df.churned == 1]
df_ = pd.concat([dft, dff.sample(dft.shape[0])])
df_ = df_.values

From now on, I repeat the same exact steps that I followed above.

In [ ]:
X = np.array(df_[:,:-1])
y = np.array(df_[:,-1])
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
y_train = y_train.astype('int')

**Logistic regression**

In [ ]:
logreg = LogisticRegression(solver = 'lbfgs')
logreg.fit(X_train_scaled, y_train)

In [ ]:
logreg.score(X_test_scaled, y_test.astype('int'))

In [ ]:
cvs = cross_val_score(LogisticRegression(), X_train_scaled, y_train, cv = 10)
np.mean(cvs), np.std(cvs)

In [ ]:
cm = confusion_matrix(y_test, logreg.predict(X_test_scaled))
print('recall: {}'.format(cm[1,1] / sum(cm[1])))
print('precision: {}'.format(cm[1,1] / (cm[0,1] + cm[1,1])))

Both recall and precision are good. Can be better maybe.

**Voting classifier:** ***Logistic regression + decision tree***

In [ ]:
voting = VotingClassifier([('logreg', LogisticRegression(C=100, solver='lbfgs')),
                          ('tree', DecisionTreeClassifier(max_depth=3))], voting = 'soft', flatten_transform=False)

In [ ]:
voting.fit(X_train_scaled, y_train)

In [ ]:
voting.score(X_test_scaled, y_test)

In [ ]:
cvs = cross_val_score(
    VotingClassifier([('logreg', LogisticRegression(C=100, solver='lbfgs')),
                      ('tree', DecisionTreeClassifier(max_depth=3))], voting = 'soft', flatten_transform=False),
                    X_train_scaled, y_train, cv = 10)
np.mean(cvs), np.std(cvs)

In [ ]:
cm = confusion_matrix(y_test, voting.predict(X_test_scaled))
print('recall: {}'.format(cm[1,1] / sum(cm[1])))
print('precision: {}'.format(cm[1,1] / (cm[0,1] + cm[1,1])))

They are better now.

**Random Forest**

In [ ]:
rf = RandomForestClassifier(n_estimators = 10, warm_start=True)

In [ ]:
rf.fit(X_train_scaled, y_train)

In [ ]:
rf.score(X_test_scaled, y_test)

In [ ]:
cvs = cross_val_score(RandomForestClassifier(), X_train_scaled, y_train, cv = 10)
np.mean(cvs), np.std(cvs)

In [ ]:
cm = confusion_matrix(y_test, rf.predict(X_test_scaled))
print('recall: {}'.format(cm[1,1] / sum(cm[1])))
print('precision: {}'.format(cm[1,1] / (cm[0,1] + cm[1,1])))

They are both good.

Among all the models that I tried above, random forest is the best model. Voting classifier is also as good as random forest. I could do some paramater tuning to improve the models. I will try that on random forest, because it is easier to do. I could adjust the debth of the trees, number of trees, and maybe the minimum size of leaves. I will try only one of these options; I will search for the optimum number of trees. For it takes a lot of time to run grid search and random forests, I will limit the number of trees between 5 and 50.

In [ ]:
from sklearn.model_selection import GridSearchCV

param = {'n_estimators': np.arange(5, 50, 5)}

grid = GridSearchCV(rf, param_grid = param, cv = 10)
grid.fit(X_train_scaled, y_train)

In [ ]:
grid.score(X_test_scaled, y_test)

In [ ]:
grid.best_params_.get('n_estimators')

Now let us create a random forest using the optimum number of trees, and then see the precision and recall.

In [ ]:
rf = RandomForestClassifier(n_estimators = grid.best_params_.get('n_estimators'), warm_start=True)
rf.fit(X_train_scaled, y_train)
rf.score(X_test_scaled, y_test)

In [ ]:
cm = confusion_matrix(y_test, rf.predict(X_test_scaled))
print('recall: {}'.format(cm[1,1] / sum(cm[1])))
print('precision: {}'.format(cm[1,1] / (cm[0,1] + cm[1,1])))